# Convoloutional Neural Networks

In [2]:
# imports 
import os
import sys
import torch
import numpy as np
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision.datasets import FashionMNIST

sns.set()
print(f'torch version is : {torch.__version__}')
print(f'numpy version is  : {np.__version__}')
print(f'sns version is  :{sns.__version__}')

torch version is : 1.11.0+cpu
numpy version is  : 1.21.5
sns version is  :0.11.2


In [3]:
# Let's get the data 
train_data = FashionMNIST(
   root='../basics/data',
   train=True,
   transform=ToTensor(),
   download=False 
)

test_data = FashionMNIST(
   root='../basics/data',
   train=False,
   transform=ToTensor(),
   download=False 
)

In [4]:
# Define parameters
epcohs = 5
num_classes = 10
batch_size = 100
hidden_size = 500
learning_rate = 0.01
input_size = 28 * 28

In [5]:
# Let's dataloading schemes
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [7]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [8]:
# Let's define the model 
from turtle import forward


class ConvNet(nn.Module):
    # Define the class parameters 
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.num_classes = num_classes
        # Define the layers 
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Define the dense layer 
        self.fc = nn.Linear(in_features=7*7*32, out_features=self.num_classes)

    # Let's define the forward pass
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # We need to reshape the output such that it has shape (7 * 7 * 32, )
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

# Let's initialize the model 
model = ConvNet(num_classes=num_classes).to(device)        

In [9]:
# Let's initialize the loss and optimizer 
loss = nn.CrossEntropyLoss() # For multi-class problem
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Let's define the training loop 
from tqdm import tqdm


for epoch in tqdm(range(epcohs)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Define the forward pass
        outputs = model(images)
        l = loss(outputs, labels)

        # Define the backpropagation and optimization 
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print(f'epoch is : {epoch}, loss is : {l.item()}')


 20%|██        | 1/5 [01:02<04:08, 62.19s/it]

epoch is : 0, loss is : 0.3886413872241974


 40%|████      | 2/5 [02:02<03:02, 60.81s/it]

epoch is : 1, loss is : 0.25359854102134705


 60%|██████    | 3/5 [03:00<01:59, 59.59s/it]

epoch is : 2, loss is : 0.22176824510097504


 80%|████████  | 4/5 [04:01<01:00, 60.31s/it]

epoch is : 3, loss is : 0.23749206960201263


100%|██████████| 5/5 [05:03<00:00, 60.62s/it]

epoch is : 4, loss is : 0.13605937361717224


In [12]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 90.04 %
